# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratório 4

Neste laboratório exploraremos frameworks avançados para construção de aplicações inteligentes: **Semantic Kernel** e **AutoGen** com Azure OpenAI. Aprenderemos como criar agentes conversacionais, orchestrar múltiplos agentes, implementar plugins personalizados e desenvolver workflows complexos de automação.

O **Semantic Kernel** é um SDK open-source da Microsoft que permite integrar modelos de IA (como Azure OpenAI) com linguagens de programação convencionais, oferecendo funcionalidades como plugins, planejamento e memória.

O **AutoGen** é um framework da Microsoft para criar aplicações multi-agente onde diferentes agentes podem colaborar para resolver problemas complexos através de conversação estruturada.

### Pré-requisitos

Certifique-se de que as variáveis de ambiente estão configuradas no arquivo `.env` na raiz do repositório com suas credenciais do Azure OpenAI.

### Exercício 1 - Configuração Inicial e Semantic Kernel

Vamos começar instalando e importando as bibliotecas necessárias para trabalhar com Semantic Kernel e AutoGen.

In [ ]:
# Instalação das dependências necessárias
#%pip install semantic-kernel pyautogen python-dotenv openai

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins import TextPlugin
from semantic_kernel.functions import KernelArguments
from semantic_kernel.prompt_template import PromptTemplateConfig
import autogen

# Carregando variáveis de ambiente
load_dotenv(dotenv_path="../../.env")

In [ ]:
# Configuração das credenciais do Azure OpenAI
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("Configurações carregadas:")
print(f"Endpoint: {azure_endpoint}")
print(f"Deployment: {deployment_name}")
print(f"API Version: {api_version}")

#### Configurando o Semantic Kernel

O Semantic Kernel atua como uma camada de orquestração que permite combinar modelos de IA com código convencional. Vamos criar um kernel e configurá-lo para usar o Azure OpenAI:

In [ ]:
# Inicializando o Semantic Kernel
kernel = sk.Kernel()

# Adicionando o serviço de chat do Azure OpenAI
chat_service = AzureChatCompletion(
    deployment_name=deployment_name,
    endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

kernel.add_service(chat_service)

print("Semantic Kernel configurado com sucesso!")

#### Primeiro Exemplo: Função Simples com Semantic Kernel

Vamos criar uma função simples que utiliza o Semantic Kernel para gerar conteúdo:

In [ ]:
# Criando uma função simples usando template de prompt
prompt_template = """
Você é um especialista em {{$topic}}. 
Crie um resumo informativo e conciso sobre {{$topic}} em {{$language}}.
O resumo deve ter aproximadamente {{$length}} palavras.

Tópico: {{$topic}}
"""

# Criando a função do kernel
summary_function = kernel.add_function(
    function_name="CreateSummary",
    plugin_name="ContentPlugin",
    prompt=prompt_template,
    description="Cria um resumo sobre qualquer tópico"
)

# Executando a função
async def run_summary_example():
    arguments = KernelArguments(
        topic="Inteligência Artificial",
        language="português", 
        length="150"
    )
    
    result = await kernel.invoke(summary_function, arguments)
    return result

# Executando o exemplo
result = await run_summary_example()
print("Resumo gerado pelo Semantic Kernel:")
print(result)

### Exercício 2 - Plugins e Funcionalidades Avançadas

#### Trabalhando com Plugins Built-in

O Semantic Kernel oferece plugins integrados que fornecem funcionalidades comuns. Vamos explorar o TextPlugin:

In [ ]:
# Adicionando plugin de texto built-in
kernel.add_plugin(TextPlugin(), plugin_name="text")

# Exemplo de uso do plugin de texto
text_to_process = """
O Azure AI Foundry é uma plataforma completa para desenvolvimento de aplicações de IA. 
Ela oferece ferramentas para treinar, implantar e gerenciar modelos de machine learning. 
Com integração nativa ao Azure OpenAI, desenvolvedores podem criar aplicações inteligentes rapidamente.
"""

async def demonstrate_text_plugin():
    # Função para fazer uppercase
    uppercase_result = await kernel.invoke(
        plugin_name="text",
        function_name="uppercase", 
        input=text_to_process
    )
    
    print("Texto em maiúsculas:")
    print(uppercase_result)
    print("\n" + "="*50 + "\n")
    
    # Função para contar palavras
    word_count = await kernel.invoke(
        plugin_name="text",
        function_name="length",
        input=text_to_process
    )
    
    print(f"Número de caracteres: {word_count}")

await demonstrate_text_plugin()

#### Criando um Plugin Personalizado

Vamos criar um plugin personalizado que simula operações matemáticas e de análise de dados:

In [ ]:
from semantic_kernel.functions import kernel_function
from typing import Annotated
import statistics

class MathPlugin:
    """Plugin personalizado para operações matemáticas"""
    
    @kernel_function(
        description="Calcula a média de uma lista de números",
        name="calculate_average"
    )
    def calculate_average(
        self, 
        numbers: Annotated[str, "Lista de números separados por vírgula"]
    ) -> Annotated[str, "A média dos números"]:
        try:
            num_list = [float(x.strip()) for x in numbers.split(',')]
            average = statistics.mean(num_list)
            return f"A média dos números {numbers} é: {average:.2f}"
        except Exception as e:
            return f"Erro ao calcular média: {str(e)}"
    
    @kernel_function(
        description="Encontra o maior número em uma lista",
        name="find_maximum"
    )
    def find_maximum(
        self, 
        numbers: Annotated[str, "Lista de números separados por vírgula"]
    ) -> Annotated[str, "O maior número"]:
        try:
            num_list = [float(x.strip()) for x in numbers.split(',')]
            maximum = max(num_list)
            return f"O maior número em {numbers} é: {maximum}"
        except Exception as e:
            return f"Erro ao encontrar máximo: {str(e)}"

# Adicionando o plugin personalizado ao kernel
math_plugin = MathPlugin()
kernel.add_plugin(math_plugin, plugin_name="math")

# Testando o plugin personalizado
async def test_custom_plugin():
    numbers = "10, 25, 15, 30, 8, 22"
    
    # Calculando média
    avg_result = await kernel.invoke(
        plugin_name="math",
        function_name="calculate_average",
        numbers=numbers
    )
    print(avg_result)
    
    # Encontrando máximo
    max_result = await kernel.invoke(
        plugin_name="math", 
        function_name="find_maximum",
        numbers=numbers
    )
    print(max_result)

await test_custom_plugin()

### Exercício 3 - Introdução ao AutoGen

O AutoGen permite criar sistemas multi-agente onde diferentes agentes podem colaborar para resolver problemas complexos. Vamos criar um exemplo básico com dois agentes: um assistente e um revisor.

#### Configurando o AutoGen

In [ ]:
# Configuração para AutoGen com Azure OpenAI
config_list = [
    {
        "model": deployment_name,
        "api_type": "azure",
        "api_base": azure_endpoint,
        "api_key": api_key,
        "api_version": api_version
    }
]

# Configuração padrão para LLM
llm_config = {
    "config_list": config_list,
    "temperature": 0.7
}

print("Configuração do AutoGen criada com sucesso!")

#### Criando Agentes Conversacionais

Vamos criar um sistema com múltiplos agentes especializados:

In [ ]:
# Criando agentes especializados
escritor = autogen.AssistantAgent(
    name="Escritor",
    system_message="""Você é um escritor especializado em criar conteúdo técnico sobre tecnologia.
    Sua função é criar artigos informativos, claros e bem estruturados.
    Sempre escreva em português brasileiro.""",
    llm_config=llm_config
)

revisor = autogen.AssistantAgent(
    name="Revisor",
    system_message="""Você é um revisor especializado em conteúdo técnico.
    Sua função é revisar textos, corrigir erros, melhorar a clareza e sugerir melhorias.
    Seja construtivo em suas críticas e sempre explique suas sugestões.
    Sempre responda em português brasileiro.""",
    llm_config=llm_config
)

usuario = autogen.UserProxyAgent(
    name="Usuario",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("FINALIZADO"),
    code_execution_config={"work_dir": "temp"},
    llm_config=llm_config
)

print("Agentes criados:")
print("- Escritor: Especialista em criar conteúdo técnico")
print("- Revisor: Especialista em revisar e melhorar textos") 
print("- Usuario: Proxy para interação humana")

In [ ]:
# Iniciando uma conversação entre agentes
task = """
Escreva um artigo de 200 palavras sobre as vantagens do Azure AI Foundry para desenvolvedores.
O artigo deve incluir:
1. Introdução ao Azure AI Foundry
2. Principais benefícios para desenvolvedores
3. Exemplos de uso prático
4. Conclusão

Após escrever o artigo, peça ao revisor para analisar e sugerir melhorias.
Quando estiver satisfeito com o resultado final, termine com a palavra FINALIZADO.
"""

# Executando a conversação
chat_result = usuario.initiate_chat(
    escritor,
    message=task
)

### Exercício 4 - Group Chat e Colaboração Avançada

#### Criando um Group Chat com Multiple Agentes

Vamos criar um cenário mais complexo onde múltiplos agentes colaboram em uma discussão em grupo:

In [ ]:
# Criando agentes especializados para discussão em grupo
analista_negocio = autogen.AssistantAgent(
    name="Analista_Negocio",
    system_message="""Você é um analista de negócios especializado em IA.
    Foque nos aspectos comerciais, ROI, e impacto no negócio.
    Sempre considere viabilidade econômica e prazos de implementação.""",
    llm_config=llm_config
)

arquiteto_solucoes = autogen.AssistantAgent(
    name="Arquiteto_Solucoes", 
    system_message="""Você é um arquiteto de soluções de IA.
    Foque na arquitetura técnica, escalabilidade, segurança e integração.
    Considere sempre as melhores práticas de engenharia.""",
    llm_config=llm_config
)

gerente_projeto = autogen.AssistantAgent(
    name="Gerente_Projeto",
    system_message="""Você é um gerente de projeto especializado em IA.
    Foque no planejamento, cronograma, recursos e gestão de riscos.
    Mantenha a discussão focada e produtiva.""",
    llm_config=llm_config
)

# Criando o Group Chat Manager
group_chat = autogen.GroupChat(
    agents=[analista_negocio, arquiteto_solucoes, gerente_projeto, usuario],
    messages=[],
    max_round=12
)

manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

print("Group Chat configurado com os seguintes participantes:")
print("- Analista de Negócios: Foco em aspectos comerciais")
print("- Arquiteto de Soluções: Foco na arquitetura técnica") 
print("- Gerente de Projeto: Foco no planejamento e execução")
print("- Manager: Coordena a discussão")

In [ ]:
# Iniciando discussão em grupo
discussao = """
Nossa empresa quer implementar um chatbot inteligente para atendimento ao cliente usando Azure AI Foundry.
O chatbot deve:
- Responder perguntas sobre produtos
- Escalar para humanos quando necessário
- Integrar com nosso sistema CRM
- Suportar português e inglês
- Ter análise de sentimento

Discutam os seguintes pontos:
1. Viabilidade técnica e comercial
2. Arquitetura recomendada
3. Cronograma de implementação
4. Principais riscos e como mitigá-los

Cada um deve contribuir com sua perspectiva especializada.
"""

# Executando a discussão em grupo
group_result = usuario.initiate_chat(
    manager,
    message=discussao
)

### Exercício 5 - Integração Avançada: Semantic Kernel + AutoGen

#### Combinando o Poder de Ambos os Frameworks

Vamos criar um exemplo que combina as capacidades do Semantic Kernel (plugins e orchestração) com o AutoGen (multi-agente). Criaremos um agente que utiliza funções do Semantic Kernel:

In [ ]:
# Criando um agente híbrido que usa Semantic Kernel
class SemanticKernelAgent:
    def __init__(self, kernel):
        self.kernel = kernel
    
    async def process_with_sk(self, task, plugin_name, function_name, **kwargs):
        """Processa uma tarefa usando funções do Semantic Kernel"""
        try:
            result = await self.kernel.invoke(
                plugin_name=plugin_name,
                function_name=function_name,
                **kwargs
            )
            return str(result)
        except Exception as e:
            return f"Erro ao executar função do SK: {str(e)}"

# Instanciando o agente híbrido
sk_agent = SemanticKernelAgent(kernel)

# Criando um agente AutoGen que utiliza o Semantic Kernel
agente_analitico = autogen.AssistantAgent(
    name="Agente_Analitico",
    system_message="""Você é um agente analítico especializado em processar dados.
    Quando precisar fazer cálculos matemáticos, você pode solicitar ao usuário para executar
    funções específicas do Semantic Kernel.
    Sempre explique suas análises de forma clara e didática.""",
    llm_config=llm_config
)

# Exemplo de integração
async def exemplo_integracao():
    print("=== Exemplo de Integração SK + AutoGen ===")
    
    # Dados para análise
    vendas_dados = "120, 150, 95, 200, 175, 130, 190"
    
    # Usando Semantic Kernel para calcular estatísticas
    media_vendas = await sk_agent.process_with_sk(
        task="calcular média",
        plugin_name="math",
        function_name="calculate_average", 
        numbers=vendas_dados
    )
    
    maximo_vendas = await sk_agent.process_with_sk(
        task="encontrar máximo",
        plugin_name="math",
        function_name="find_maximum",
        numbers=vendas_dados
    )
    
    print("Resultados do Semantic Kernel:")
    print(f"- {media_vendas}")
    print(f"- {maximo_vendas}")
    
    # Agora vamos usar o AutoGen para interpretar os resultados
    relatorio = f"""
    Analise os seguintes dados de vendas e forneça insights:
    
    Dados de vendas (em milhares): {vendas_dados}
    {media_vendas}
    {maximo_vendas}
    
    Forneça uma análise com:
    1. Interpretação dos números
    2. Tendências observadas
    3. Recomendações para melhorar vendas
    """
    
    # Iniciando chat com o agente analítico
    chat_result = usuario.initiate_chat(
        agente_analitico,
        message=relatorio,
        max_turns=2
    )

await exemplo_integracao()

### Melhores Práticas e Casos de Uso Avançados

#### Quando Usar Semantic Kernel vs AutoGen

**Semantic Kernel é ideal para:**
- Orchestração de AI com código tradicional
- Criação de plugins e funções reutilizáveis
- Aplicações que precisam de planejamento automático
- Integração com sistemas existentes
- Aplicações single-agent com funcionalidades complexas

**AutoGen é ideal para:**
- Sistemas multi-agente complexos
- Simulações de discussões e brainstorming
- Workflows que envolvem múltiplas perspectivas
- Automação de processos colaborativos
- Sistemas de tomada de decisão distribuída

#### Casos de Uso Práticos

1. **Sistema de Atendimento ao Cliente**
   - Semantic Kernel: Criação de plugins para consulta de pedidos, produtos, etc.
   - AutoGen: Agentes especializados (triagem, vendas, suporte técnico)

2. **Análise de Documentos Corporativos**
   - Semantic Kernel: Funções para extração e processamento de texto
   - AutoGen: Agentes revisores, analistas e especialistas em domínio

3. **Planejamento de Projetos**
   - Semantic Kernel: Plugins para cálculos de cronograma e recursos
   - AutoGen: Agentes representando diferentes stakeholders

4. **Sistema de Recomendações**
   - Semantic Kernel: Funções de filtragem e cálculo de similaridade
   - AutoGen: Agentes com diferentes critérios de recomendação

In [ ]:
# Exemplo Final: Sistema de Análise de Produtos
# Combinando SK e AutoGen para análise de feedback de produtos

# Plugin do SK para análise de sentimentos (simulado)
class SentimentPlugin:
    @kernel_function(
        description="Analisa o sentimento de um texto",
        name="analyze_sentiment"
    )
    def analyze_sentiment(
        self, 
        text: Annotated[str, "Texto para análise de sentimento"]
    ) -> Annotated[str, "Resultado da análise de sentimento"]:
        # Simulação simples de análise de sentimento
        palavras_positivas = ["excelente", "ótimo", "bom", "recomendo", "perfeito"]
        palavras_negativas = ["ruim", "péssimo", "horrível", "não recomendo", "defeito"]
        
        text_lower = text.lower()
        score_positivo = sum(1 for palavra in palavras_positivas if palavra in text_lower)
        score_negativo = sum(1 for palavra in palavras_negativas if palavra in text_lower)
        
        if score_positivo > score_negativo:
            return f"POSITIVO (Score: +{score_positivo - score_negativo})"
        elif score_negativo > score_positivo:
            return f"NEGATIVO (Score: -{score_negativo - score_positivo})"
        else:
            return "NEUTRO (Score: 0)"

# Adicionando o plugin ao kernel
sentiment_plugin = SentimentPlugin()
kernel.add_plugin(sentiment_plugin, plugin_name="sentiment")

# Criando agentes especializados para análise de produtos
agente_qualidade = autogen.AssistantAgent(
    name="Especialista_Qualidade",
    system_message="""Você é um especialista em qualidade de produtos.
    Analise feedbacks focando em aspectos de qualidade, durabilidade e design.
    Identifique padrões e sugira melhorias.""",
    llm_config=llm_config
)

agente_experiencia = autogen.AssistantAgent(
    name="Especialista_UX",
    system_message="""Você é um especialista em experiência do usuário.
    Analise feedbacks focando na usabilidade, facilidade de uso e satisfação.
    Sugira melhorias na experiência do cliente.""",
    llm_config=llm_config
)

# Exemplo de execução do sistema completo
async def sistema_analise_produtos():
    feedbacks = [
        "O produto é excelente! Muito fácil de usar e a qualidade surpreendeu.",
        "Tive problemas com o produto. Não funciona como esperado, péssima experiência.",
        "O design é bom mas poderia melhorar a usabilidade. Recomendo com ressalvas."
    ]
    
    print("=== Sistema de Análise de Produtos ===\n")
    
    # Análise de sentimentos usando Semantic Kernel
    analises_sentimento = []
    for i, feedback in enumerate(feedbacks, 1):
        sentimento = await kernel.invoke(
            plugin_name="sentiment",
            function_name="analyze_sentiment",
            text=feedback
        )
        analises_sentimento.append(f"Feedback {i}: {sentimento}")
        print(f"Feedback {i}: {feedback}")
        print(f"Sentimento: {sentimento}\n")
    
    # Preparando relatório para os agentes
    relatorio_consolidado = f"""
    Análise de Feedbacks de Produtos - Relatório Consolidado
    
    Feedbacks analisados:
    {chr(10).join(f'{i+1}. {feedback}' for i, feedback in enumerate(feedbacks))}
    
    Análises de sentimento:
    {chr(10).join(analises_sentimento)}
    
    Por favor, forneçam suas análises especializadas e recomendações para melhorar o produto.
    """
    
    # Criando discussão entre especialistas
    group_chat_produtos = autogen.GroupChat(
        agents=[agente_qualidade, agente_experiencia, usuario],
        messages=[],
        max_round=8
    )
    
    manager_produtos = autogen.GroupChatManager(
        groupchat=group_chat_produtos,
        llm_config=llm_config
    )
    
    # Iniciando discussão
    chat_result = usuario.initiate_chat(
        manager_produtos,
        message=relatorio_consolidado
    )

# Executar o sistema (descomente para testar)
# await sistema_analise_produtos()

print("Sistema de análise de produtos configurado!")
print("Descomente a última linha para executar o exemplo completo.")

### Conclusão e Próximos Passos

Neste laboratório, exploramos as poderosas capacidades do **Semantic Kernel** e **AutoGen** para criar aplicações inteligentes avançadas com Azure OpenAI. Aprendemos:

#### Semantic Kernel
- ✅ Configuração e inicialização do kernel
- ✅ Criação e uso de funções personalizadas
- ✅ Trabalho com plugins built-in e personalizados
- ✅ Templates de prompts e argumentos dinâmicos

#### AutoGen
- ✅ Configuração de agentes conversacionais
- ✅ Criação de sistemas multi-agente
- ✅ Group Chat e colaboração entre agentes
- ✅ Especialização de agentes por domínio

#### Integração Avançada
- ✅ Combinação de SK e AutoGen
- ✅ Sistemas híbridos com múltiplas capacidades
- ✅ Casos de uso práticos e empresariais

### Recursos Adicionais

Para aprofundar seus conhecimentos, explore os seguintes recursos:

#### Semantic Kernel
- [Documentação Oficial do Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Repositório GitHub do Semantic Kernel](https://github.com/microsoft/semantic-kernel)
- [Exemplos e Tutoriais](https://learn.microsoft.com/en-us/semantic-kernel/get-started/)
- [Plugins da Comunidade](https://github.com/microsoft/semantic-kernel/tree/main/python/semantic_kernel/core_plugins)

#### AutoGen
- [Documentação Oficial do AutoGen](https://microsoft.github.io/autogen/)
- [Repositório GitHub do AutoGen](https://github.com/microsoft/autogen)
- [Galeria de Exemplos](https://microsoft.github.io/autogen/docs/Examples)
- [Notebooks de Tutorial](https://github.com/microsoft/autogen/tree/main/notebook)

#### Azure AI Foundry
- [Portal do Azure AI Foundry](https://ai.azure.com/)
- [Documentação do Azure OpenAI](https://learn.microsoft.com/en-us/azure/ai-services/openai/)
- [Modelos Disponíveis](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/foundry-models-overview)
- [Melhores Práticas](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/best-practices)

### Desafios para Prática

1. **Expansão do Sistema de Produtos**: Adicione mais plugins ao Semantic Kernel para análise de imagens de produtos
2. **Agentes Especializados**: Crie agentes para diferentes setores (finanças, saúde, educação)
3. **Workflow Complexo**: Implemente um sistema de aprovação multi-etapas usando AutoGen
4. **Integração External**: Conecte os frameworks com APIs externas e bases de dados
5. **Interface de Usuário**: Crie uma interface web para interagir com seus agentes